In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('../input/chinese-mnist-digit-recognizer/chineseMNIST.csv')

In [ ]:
df.head()

In [ ]:
df.to_numpy().shape

In [ ]:
df['label'].replace(100,11,inplace=True)
df['label'].replace(1000,12,inplace=True)
df['label'].replace(10000,13,inplace=True)
df['label'].replace(100000000,14,inplace=True)

In [ ]:
labels = df['label']

In [ ]:
df.drop(['label'], axis=1, inplace=True)
df.drop(['character'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
dataset = df.to_numpy()

In [ ]:
labels = labels.to_numpy()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42)

In [ ]:
x_test.shape

In [ ]:
x_train = x_train.reshape(12000,64,64,1)
x_test = x_test.reshape(3000,64,64,1)

print(x_test.shape)

In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [ ]:
plt.imshow(x_train[6])
x_train.shape

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import InputLayer, Dense, Conv2D, BatchNormalization, Dropout, Flatten,LeakyReLU,MaxPool2D
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam

In [ ]:
import keras_tuner as kt

In [ ]:
def create_model(hp):
    model = keras.Sequential()
    model.add(InputLayer(input_shape=(64,64,1)))
    model.add(Conv2D(hp.Int("input_units",32,128,32),3,padding='same',activation='relu'))
    model.add(MaxPool2D())
    model.add(BatchNormalization())
    drop = hp.Choice('Dropout_rate',values=[0.1,0.2,0.3])
    model.add(Dropout(drop))
    
    for i in range(hp.Int('number_of_layers',1,3,1)):
        model.add(Conv2D(hp.Int("input_units1",64,256,32),3,padding='same',activation='relu'))

    model.add(MaxPool2D())
    model.add(BatchNormalization())
    model.add(Dropout(drop))
    model.add(Flatten())
    model.add(Dense(hp.Int('input_units2',512,1024,128),activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(drop))
    model.add(Dense(15))

    loss = SparseCategoricalCrossentropy(from_logits=True)
    opt = Adam(hp.Choice('lr',values=[1e-2, 1e-3, 1e-4]))

    model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])
    return model
    


In [ ]:
tuner = kt.Hyperband(create_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='chinese_mnist')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete.
The optimal number of input units for the first Conv2D layer is: {best_hps.get('input_units')},\n
The optimal dropout rate is: {best_hps.get('Dropout_rate')},\n
The optimal number of Conv layers after the first one is: {best_hps.get('number_of_layers')},\n
The optimal number of input units for those Conv2D layers is: {best_hps.get('input_units1')},\n
The optimal number of input units for Dense layer is: {best_hps.get('input_units2')},\n
The optimal learning rate is: {best_hps.get('lr')},\n
""")

In [ ]:
model = tuner.hypermodel.build(best_hps)

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train,y_train,epochs=20,validation_split = 0.2,batch_size=128)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
results = model.evaluate(x_test,y_test,batch_size=64)
print("test loss, test acc:", results)

In [ ]:
predictions = model.predict(x_test)


In [ ]:
subs = []
for p in predictions:
    subs.append(p.argmax())

In [ ]:
results_df = pd.DataFrame({'ImageId' : range(1,3001), 'Label' : list(subs)})
results_df.head(10)
results_df.shape

In [ ]:
results_df.to_csv("results.csv",index=False)